In [ ]:
#from google.colab import drive
#drive.mount("/content/drive")

In [ ]:
from os.path import join
from numpy import reshape,newaxis,where,nan,load,array,log10,power,min,max,arange
from pandas import read_csv

Dat = "path/to/data"

Case = load(join(Dat,"an-VorZ-interp2d.npy"))[:,:,:,newaxis]

<h5>
===================         </br>
Recall CNN-AE Trained Model </br>
===================         </br>
</h>

In [ ]:
from keras.models import load_model,Model
a2f = "path/to/model"
a2 = load_model(join(a2f,"VORZ-AE-7-3-ADAM-1"))

<h5>
======================            </br>
Extract Encoder and Decoder Parts </br>
======================            </br>
get_enc: From input to Enc layer  </br>
get_dec: From Enc to output layer </br>

=======  </br>
STRATEGY </br>
=======  </br>
(1) Collect Reduced Data at Enc layer enc_data (140 samples) </br>
(2) Provide Normalized Train Data (Ra,Re) <-> (X,y) enc_data </br>
(3) Split Random-Seed Trn and Tst of (2) by 80% - 20% </br>
(4) Fit enc_model to construct Reduced Data (Enc) model f(Ra,Re) </br>
(5) Upscaling Reduced Data to Flow Field T(Ra,Re) = get_dec * f(Ra,Re) </br>
</h5>

In [ ]:
# Encoder: Between Input and Enc
get_enc = Model(inputs=a2.input, outputs=a2.get_layer("max_pooling2d_115").output) 
# Decoder: Between Enc and Ouptut
get_dec = Model(inputs=a2.get_layer("max_pooling2d_115").output, outputs=a2.get_layer("conv2d_260").output) 


<h5>
================        </br>
Training Encoder Model  </br>
================        </br>
</h>

In [ ]:
enc_data = get_enc.predict(Case)
enc_data = enc_data.reshape((len(Case), 8*4*16))
print("Original dimension = ",(Case.shape[0],Case.shape[1]*Case.shape[2]))
print("Encoded dimension  = ",enc_data.shape)

##### (A) Transform Reduced data (Ra,Re)(enc_data) -> (Log,MinMax)(MinMax)

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler,MaxAbsScaler,power_transform
from numpy import log,log10,power,exp

def log10trn(X):    return log10(X)
def invlog10trn(X): return power(10,X)

def scaledLog10(X):
    x = 9*(X-X.min())/(X.max()-X.min()) + 1
    return log10(x)

def invScaledLog10(X):
    x = power(10,X)
    return (X-1)*(X.max()-X.min())/9+X.min()

X = array(read_csv(join(Dat,"an-features.csv")))
y = enc_data

print(y.max(),y.min())

Xscale1 = FunctionTransformer(func=log10trn,inverse_func=invlog10trn,validate=True)
Xscale2 = MinMaxScaler()
yscale  = MinMaxScaler(feature_range=(0,0.1))
#yscale  = FunctionTransformer(func=scaledLog10,inverse_func=invScaledLog10,validate=True)

X[:,0] = Xscale1.transform(X[:,0].reshape(-1,1)).flatten()
X[:,1] = Xscale2.fit_transform(X[:,1].reshape(-1,1)).flatten()

y      = yscale.fit_transform(y)
#y      = yscale.transform(y)

XMIN = X.min(axis=0)
XMAX = X.max(axis=0)

print("X shape = ",X.shape)
print("y shape = ",y.shape)

print("X1 (Ra) min, max = ",XMIN[0],XMAX[0])
print("X2 (Re) min, max = ",XMIN[1],XMAX[1])

print(y.max(),y.min())

##### Train-Test Split by RandInt-Seed

In [ ]:
from sklearn.model_selection import train_test_split

indices = arange(0,140,dtype=int)
#X_trn,X_tst,y_trn,y_tst,idx_trn,idx_tst = train_test_split(X,y,indices,random_state=42,test_size=0.2)
#X_trn,X_tst,y_trn,y_tst,idx_trn,idx_tst = train_test_split(X,y,indices,random_state=0,test_size=0.1)
X_trn,X_tst,y_trn,y_tst,idx_trn,idx_tst = train_test_split(X,y,indices,random_state=0,test_size=0.1)

print(idx_trn.shape)
print(idx_tst.shape)
print(len(X_trn),len(y_trn))
print(len(X_tst),len(y_tst))

Train_len = idx_trn.shape[0]
Test_len = idx_tst.shape[0]

print("Train - Test Lenghts = ",Train_len,Test_len)

In [ ]:
ftr = read_csv(join(Dat,"an-features.csv")).to_numpy()
ftr_tst = ftr[idx_tst]
ff = open(join(Dat,"an2d-test-ftr.txt"),"w")
for i in range(len(idx_tst)):
    ff.writelines("Test Case = %4d, Ra = %9.0f, Re = %5.0f\n"%(idx_tst[i],ftr_tst[i,0],ftr_tst[i,1]))
ff.close()

##### Building Encoder Model

In [ ]:
from keras.layers import Input,Dense,Dropout
from keras.regularizers import l1,l1_l2,l2
from keras.optimizers import adam_v2
import tensorflow as tf
from keras.layers import LeakyReLU


input_dim  = X.shape[1]
output_dim = y.shape[1]

method1 = "relu"   #LeakyReLU(alpha=0.1) #"relu"
method2 = "linear" #LeakyReLU(alpha=0.1) #"linear"

kreg    = None # l2(1e-6) 

hidden_dm   =  [  16,    64,  256, 1024   ]
#hidden_dm   = [4, 8, 16, 32, 64, 128, 256]
#hidden_dm   = [   8,     128     ]
#hidden_dm   =  [  16,    64,  256, 1024, 2048, 4096, 2048, 1024  ]

print("Hidden-Layer size (",len(hidden_dm),") = ",hidden_dm)

#============
# Input layer 
input = Input(shape=(input_dim,),name="input") 

# Hidden Layers
for i in range(0,len(hidden_dm)):
    if i == 0: xh = input
    else: xh = x             
    x = Dense(hidden_dm[i],
              activation=method1,
              #kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.,stddev=0.1),
              bias_initializer=tf.keras.initializers.Zeros(),
              #bias_initializer='random_normal',
              #kernel_regularizer=kreg,
              kernel_initializer='random_normal',
              name="lay-"+str(i))(xh)
   
# Output Layer
output = Dense(output_dim,activation=method2,kernel_initializer='random_normal',name="output")(x)

enc_model = Model(input,output)

#============
# Optimizer
opt = adam_v2.Adam(learning_rate=0.01,decay=1e-4/1000)
#opt = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.99)

enc_model.compile(loss="mse",optimizer=opt)
enc_model.summary()

##### Controling Learning Rate (step_decay)

In [ ]:
from keras.callbacks import LearningRateScheduler
from numpy import floor,fabs,maximum,minimum

# learning rate schedules

def constant(epoch):
	lrate = 0.0001
	return lrate

def step_decay(epoch):
	initial_lrate = 0.001
	drop = 0.1
	epochs_drop = 2500 #250
	lrate = initial_lrate * pow(drop,floor((1+epoch)/epochs_drop))
	return lrate

def CLR(epoch):
	step_size = 1000
	cycle = floor(1 + epoch / ( 2 * step_size))
	x = fabs(epoch / step_size - 2 * cycle + 1 )
	#lrate = 0.2 - (0.2 - 0.02) * maximum(0.0,1.0-x)
	lrate = 0.001 + (0.001 - 0.0001) * maximum(0.0,1.0-x)
	return lrate

def expo_decay(epochs):
	initial_rate = 0.001
	decay_steps = 1000
	decay_rate  = 0.96
	lrate = initial_rate * decay_rate**(epochs/decay_steps)
	return lrate

lrate = tf.keras.callbacks.LearningRateScheduler(constant)
callbacks_list = [lrate]

##### ==> Fitting enc_model

In [ ]:
#enc_model = load_model("ENC-Mid-7-SGD-3-4")

hist = enc_model.fit(X_trn,
                     y_trn,
                     epochs=10000,
                     shuffle=True,
                     batch_size=int(X_trn.shape[0]),
                     callbacks=callbacks_list,
                     validation_data=(X_tst,y_tst),verbose=2) 

import matplotlib.pyplot as plt

plt.style.use("dark_background")
plt.xscale("log")
plt.yscale("log")
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])
plt.show()

##### Get back modeled reduced data

In [ ]:
from sklearn.metrics import mean_squared_error
pred_trn = enc_model.predict(X_trn)
pred_tst = enc_model.predict(X_tst)

avg_trn = 0
for i in range(len(X_trn)): 
    avg_trn += mean_squared_error(y_trn[i],pred_trn[i])
avg_trn /=len(X_trn)

avg_tst = 0
for i in range(len(X_tst)): 
    avg_tst += mean_squared_error(y_tst[i],pred_tst[i])
avg_tst /=len(X_tst)

print("AVERAGED MSE Error TRAIN = %.4e" % avg_trn)
print("AVERAGED MSE Error TEST = %.4e" % avg_tst)

In [ ]:
enc_model.save(join(a2f,"SC-VORZ-ENC-Mid-4s-ADAM-1"))

enc_loaded = load_model(join(a2f,"SC-VORZ-ENC-Mid-4s-ADAM-1"))
pred = enc_loaded.predict(X_tst)

<h5>
====================          </br>
Recover Deconder Data (TEST)  </br>
====================          </br>
</h>

##### (B) Inverse Transform TEST Reduced data y -> enc_data

In [ ]:
yp_tst = yscale.inverse_transform(pred_tst)
yp_tst = yp_tst.reshape((len(idx_tst), 8,4,16))

<h5>
====================           </br>
Final Test Data Reconstruction </br>
====================           </br>
</h>


In [ ]:
ypp = get_dec.predict(yp_tst)

trange = arange(0,len(idx_tst),dtype=int)

avg_tst = 0
for i,j in zip(idx_tst,trange): 
    avg_tst += mean_squared_error(Case[i,:,:,0],ypp[j,:,:,0])
avg_tst /=len(idx_tst)

print("AVERAGED MSE Error TEST = %.4e" % avg_tst)

##### Take (Re,Ra) of Test Cases

In [ ]:
# ftr = read_csv(join(Dat,"an-features.csv")).to_numpy()
# ftr_tst = ftr[idx_tst]
# ff = open(join(Dat,"an2d-test-ftr.txt"),"w")
# for i in range(len(idx_tst)):
#     ff.writelines("Test Case = %4d, Ra = %9.0f, Re = %5.0f\n"%(idx_tst[i],ftr_tst[i,0],ftr_tst[i,1]))
# ff.close()

#### CASE CHECK

In [ ]:
#CASE_CHECK = [5000,150]
#CASE_CHECK = [400000,300]
#CASE_CHECK = [200000,600]
CASE_CHECK = [900000,300]

try:
  get_case = int(where((ftr == CASE_CHECK).all(axis=1))[0])
  print("Test Case in Case = ",get_case)
  get_pred = int(where((ftr_tst == CASE_CHECK).all(axis=1))[0])
  print("Test Case in idx_tst = ",get_pred)
except:
    print("-> Case Chech ",CASE_CHECK," is not in the test list")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure,show,contourf
from numpy import linspace,logspace,meshgrid,cos,sin,pi

rad = linspace(0.25,0.5,64)
tet = linspace(0,2.0*pi,128)

rmat,tmat = meshgrid(rad,tet)

xi = 0.5 + rmat * cos(tmat)
yi = 0.5 + rmat * sin(tmat)

yc = Case[get_case][:,:,0]
yp = ypp[get_pred][:,:,0]
print("Precidtion max = ",yp.max(),", min = ",yp.min())
print(yc.shape,yp.shape)

Fig = figure(figsize=(5,5))
ccf = contourf(xi,yi,yc,cmap="jet")
Fig.show()

Fig = figure(figsize=(5,5))
ccf = contourf(xi,yi,yp,cmap="jet")
Fig.show()

Fig = figure(figsize=(6,5))
ccf = contourf(xi,yi,yp-yc,cmap="jet")
plt.colorbar(cmap="jet")
Fig.show()